# Text Detection using DB++

## 1. Project clone and Environment preparation

In [2]:
!git clone https://github.com/PaddlePaddle/PaddleOCR

Cloning into 'PaddleOCR'...
remote: Enumerating objects: 46135, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 46135 (delta 0), reused 2 (delta 0), pack-reused 46130
Receiving objects: 100% (46135/46135), 339.18 MiB | 1.42 MiB/s, done.
Resolving deltas: 100% (32445/32445), done.


In [3]:
%cd PaddleOCR

/home/theanh/workspace/orai/real_estate_information_matching/PaddleOCR


In [4]:
!ls

LICENSE      README_ch.md  benchmark  paddleocr.py	setup.py
MANIFEST.in  StyleText	   configs    ppocr		test_tipc
PPOCRLabel   __init__.py   deploy     ppstructure	tools
README.md    applications  doc	      requirements.txt	train.sh


In [7]:
!pip install --quiet -r requirements.txt

In [11]:
!pip install --quiet paddlepaddle

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.2 which is incompatible.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.0 which is incompatible.
onnx 1.13.0 requires protobuf<4,>=3.20.2, but you have protobuf 3.20.0 which is incompatible.


In [12]:
!pip show paddlepaddle

Name: paddlepaddle
Version: 2.4.2
Summary: Parallel Distributed Deep Learning
Home-page: 
Author: 
Author-email: Paddle-better@baidu.com
License: Apache Software License
Location: /home/theanh/venv/lib/python3.10/site-packages
Requires: astor, decorator, numpy, opt-einsum, paddle-bfloat, Pillow, protobuf, requests, six
Required-by: 


## 2. Normal inference

In [13]:
!python tools/export_model.py \
    -c configs/det/det_r50_vd_db.yml \
    -o Global.pretrained_model=../assets/checkpoints/det_r50_vd_db_v2.0_train/best_accuracy \
    Global.save_inference_dir=../assets/inference/det_db

/home/theanh/venv/lib/python3.10/site-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/home/theanh/venv/lib/python3.10/site-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/home/theanh/venv/lib/python3.10/site-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declar

In [40]:
!python3 tools/infer/predict_det.py \
    --image_dir="../assets/test" \
    --det_model_dir="../assets/inference/det_db"

/home/theanh/venv/lib/python3.10/site-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/home/theanh/venv/lib/python3.10/site-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/home/theanh/venv/lib/python3.10/site-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declar

## 3. Convert model to ONNX

In [35]:
!pip install paddle2onnx -U

In [16]:
!pip install onnxruntime

In [33]:
# Convert model to ONNX
!paddle2onnx --model_dir ../assets/inference/det_db \
    --model_filename inference.pdmodel \
    --params_filename inference.pdiparams \
    --save_file ../assets/onnx/det_db/model.onnx \
    --opset_version 11 \
    --input_shape_dict="{'x':[-1,3,-1,-1]}" \
    --enable_onnx_checker True

2023-04-05 10:18:43 [WARNING]	[Deprecated] The flag `--input_shape_dict` is deprecated, if you need to modify the input shape of PaddlePaddle model, please refer to this tool https://github.com/jiangjiajun/PaddleUtils/tree/main/paddle 
[Paddle2ONNX] Start to parse PaddlePaddle model...
[Paddle2ONNX] Model file path: ../assets/inference/det_db/inference.pdmodel
[Paddle2ONNX] Paramters file path: ../assets/inference/det_db/inference.pdiparams
[Paddle2ONNX] Start to parsing Paddle model...
[Paddle2ONNX] Use opset_version = 11 for ONNX export.
[Paddle2ONNX] PaddlePaddle model is exported as ONNX format now.
2023-04-05 10:18:44 [INFO]	===============Make PaddlePaddle Better!================
2023-04-05 10:18:44 [INFO]	A little survey: https://iwenjuan.baidu.com/?code=r8hu2s


In [19]:
# visualize onnx model
!pip install netron

In [36]:
!netron

Serving at http://localhost:8080
^C

Stopping http://localhost:8080


## 4. ONNX Inference

In [34]:
!python3 tools/infer/predict_det.py \
    --image_dir="../assets/test/example.jpg" \
    --det_model_dir="../assets/onnx/det_db/model.onnx" \
    --use_onnx=True \
    --use_gpu=False

/home/theanh/venv/lib/python3.10/site-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/home/theanh/venv/lib/python3.10/site-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/home/theanh/venv/lib/python3.10/site-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declar

## 5. Custom inference

In [41]:
import os
import sys
os.path.abspath(__file__)

NameError: name '__file__' is not defined